In [1]:
import os
os.system("sudo apt-get update")
os.system("sudo apt-get install -y python3-opencv")
os.system("pip install opencv-python")
os.system("pip install matplotlib")
os.system("pip install pandas")
import sys
sys.path.append("..")
import cv2
import numpy as np
from utils.imutils import jimshow as show
from utils.imutils import jimshow_channel as show_channel
import matplotlib.pyplot as plt
import pandas as pd

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Fetched 229 kB in 1s (233 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
python3-opencv is already the newest version (4.5.4+dfsg-9ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


/tmp/ipykernel_33156/3284280671.py:14: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#function for reading in a file and normailzing it
def normit(image):
    hist = cv2.calcHist([image],[0,1,2],None,[255,255,255],[0,256,0,256,0,256])
    normalized_hist = cv2.normalize(hist, hist, 0, 1.0, cv2.NORM_MINMAX)
    return(normalized_hist)

# loop load all flowers and names and save it
fp = "/work/cds-vis-data/flowers/"
fp_s = os.listdir(fp)


image_list = []
name_list = []
for i in fp_s:
    filepath = fp + i
    image = cv2.imread(filepath)
    names = i
    name_list.append(i)
    image_list.append(image)





In [21]:
def count_hist_dist(filename,name_list,image_list):

    #find target according file index
    target_ind = name_list.index(filename) 
    #create target comparison hist
    target = normit(image_list[target_ind])

    #compare to target
    comp_list =[]
    for i in image_list[:10]:
        norm =  normit(i)
        histcomp = round(cv2.compareHist(target,norm,cv2.HISTCMP_CHISQR), 2)
        #save comparison values
        comp_list.append(histcomp)

    #assign filenames to comparison values

    df_pd = pd.DataFrame({"Filename":name_list[:10]})
    df_pd["Distance"] = comp_list

    #sort and filter
    #sort ascending
    df_sorted = df_pd.sort_values(axis=0, by= "Distance", ascending= True)
    #drops 1st entry (it's the target image themselves)
    df_sorted = df_pd.drop(axis=0, index=0)
    #keeps 5 closest images
    df_fin = df_sorted[:5]

    return(df_fin)


,Filename,Distance
1,image_0533.jpg,1427.97
2,image_0943.jpg,1924.50
3,image_1187.jpg,5251.09
4,image_1305.jpg,1792.27
5,image_1340.jpg,1473.92


In [ ]:
#make it into function
#turn off [:10] limit
